# Tabular data

> Utilities to process remote sensing image data into tabular format

In [ ]:
#| default_exp data.tabular

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fastcore.basics import *
import rasterio as rio
import pandas as pd
import numpy as np
import geopandas as gpd
import logging
from rasterstats import zonal_stats
from pathlib import Path

In [ ]:
example_data_path= Path('example_data')
example_points = example_data_path/'points/points.geojson'
example_polys = example_data_path/'polygons/polygons.geojson'
example_raster = example_data_path/'s2_lataseno_ex.tif'

## Data conversion and processing

Utility functions to process dataframes.

In [ ]:
#| export

def array_to_longform(a:pd.DataFrame, columns:list) -> pd.DataFrame:
    "Convert pd.DataFrame `a` to longform array"
    dfa = pd.DataFrame(data=a, columns=columns)
    dfa = dfa.reset_index()
    return dfa.melt(id_vars=["index"], ignore_index=False)

In [ ]:
#| export

def drop_small_classes(df:pd.DataFrame, min_class_size:int, target_column:str|int=0) -> pd.DataFrame:
    "Drop rows from the dataframe if their `target_column` value has less instances than `min_class_size"
    if type(target_column) == int:
        target = df.columns[0]
    else:
        target = target_column
    drop_classes = df[target].value_counts()[df[target].value_counts() < min_class_size].index.values
    drop_series = ~df[target].isin(drop_classes)
    logging.info(f'Classes with less than {min_class_size} were dropped:')
    logging.info(drop_classes)
    return df.loc[drop_series, :]

Generate random data to test.

In [ ]:
ex_df = pd.DataFrame({'label': np.random.randint(1, 10, 200)})
ex_df.label.value_counts()

label
4    32
3    23
2    22
9    21
1    21
6    21
5    21
8    21
7    18
Name: count, dtype: int64

Column name can be either specified with string or int. If not provided it defaults to first column.

In [ ]:
filtered = drop_small_classes(ex_df, 20, 'label')
assert filtered.label.value_counts().min() >= 20
filtered.label.value_counts()

label
4    32
3    23
2    22
9    21
1    21
6    21
5    21
8    21
Name: count, dtype: int64

If not specified, defaults to first column.

In [ ]:
filtered = drop_small_classes(ex_df, 20)
filtered.label.value_counts()

label
4    32
3    23
2    22
9    21
1    21
6    21
5    21
8    21
Name: count, dtype: int64

## Sampling utilities

These functions enable sampling of raster values using either point or polygon features. 

In [ ]:
#| export

def sample_raster_with_points(sampling_locations:Path, 
                              input_raster:Path, 
                              target_column:str,
                              gpkg_layer:str=None,
                              band_names:list[str]=None, 
                              rename_target:str=None) -> gpd.GeoDataFrame:
    "Extract values from `input_raster` using points from `sampling_locations`. Returns a `gpd.GeoDataFrame` with columns `target_column`, `geometry` and bands"

    if str(sampling_locations).endswith('gpkg') and not gpkg_layer:
        raise Exception(
           '`sampling_locations` is .gpkg but no `gpkg_layer` specified'
        )
    
    gdf = gpd.read_file(sampling_locations, layer=gpkg_layer)

    with rio.open(input_raster) as src:
        if src.gcps[1]: in_crs = src.gcps[1]
        else: in_crs = src.crs
        gdf = gdf.to_crs(in_crs)
        coords = [(x,y) for x,y in zip(gdf.geometry.x, gdf.geometry.y)]
        values = np.array([p for p in src.sample(coords)])
        prof = src.profile

    bands = [f'band_{i}' for i in range_of(values[0])]

    out_gdf = gdf[[target_column, 'geometry']].copy()
    
    if band_names:
        assert len(bands) == len(band_names), f'Mismatch provided band names ({len(band_names)}) and number of bands in raster ({len(bands)})'
        bands = band_names

    if target_column in bands:
        if not rename_target:
            raise Exception(
                "One of the band names is the same as target column. Provide rename_target"
                )
        out_gdf.rename(columns={target_column: rename_target}, inplace=True)
    elif rename_target:
        out_gdf.rename(columns={target_column: rename_target}, inplace=True)
        
    for i, b in enumerate(bands):
        out_gdf[b] = values[:,i].astype(prof['dtype'])
    return out_gdf

`sample_raster_with_points` is an utility to sample point values from a raster and get the results into a `gpd.GeoDataFrame`. 

In [ ]:
#| hide

# tests
test_fail(sample_raster_with_points, args=('test.gpkg', example_raster, 'id'))
test_fail(sample_raster_with_points, args=(example_points, example_raster, 'id', None, ['id']*9))

In [ ]:
out_gdf = sample_raster_with_points(example_points, example_raster, 'id')
out_gdf.head()

,id,geometry,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8
0,10.0,POINT (311760.599 7604880.391),334,591,439,1204,2651,3072,3177,2070,1046
1,14.0,POINT (312667.464 7605426.442),183,359,282,759,1742,2002,2037,1392,669
2,143.0,POINT (313619.160 7604550.762),281,478,427,900,1976,2315,2423,2139,1069
3,172.0,POINT (311989.967 7605411.190),287,530,393,1078,2446,2761,2978,1949,950
4,224.0,POINT (313386.009 7604304.917),204,379,327,753,1524,1747,1771,1322,663


It is also possible to provide `band_names` to rename the columns.

In [ ]:
band_names = ['blue', 'green', 'red', 'red_edge1', 'red_edge2', 'nir', 'narrow_nir', 'swir1', 'swir2']
out_gdf = sample_raster_with_points(example_points, example_raster, 'id', band_names=band_names)
out_gdf.head()

,id,geometry,blue,green,red,red_edge1,red_edge2,nir,narrow_nir,swir1,swir2
0,10.0,POINT (311760.599 7604880.391),334,591,439,1204,2651,3072,3177,2070,1046
1,14.0,POINT (312667.464 7605426.442),183,359,282,759,1742,2002,2037,1392,669
2,143.0,POINT (313619.160 7604550.762),281,478,427,900,1976,2315,2423,2139,1069
3,172.0,POINT (311989.967 7605411.190),287,530,393,1078,2446,2761,2978,1949,950
4,224.0,POINT (313386.009 7604304.917),204,379,327,753,1524,1747,1771,1322,663


Or rename target column

In [ ]:
out_gdf = sample_raster_with_points(example_points, example_raster, 'id', rename_target='target')
out_gdf.head()

,target,geometry,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8
0,10.0,POINT (311760.599 7604880.391),334,591,439,1204,2651,3072,3177,2070,1046
1,14.0,POINT (312667.464 7605426.442),183,359,282,759,1742,2002,2037,1392,669
2,143.0,POINT (313619.160 7604550.762),281,478,427,900,1976,2315,2423,2139,1069
3,172.0,POINT (311989.967 7605411.190),287,530,393,1078,2446,2761,2978,1949,950
4,224.0,POINT (313386.009 7604304.917),204,379,327,753,1524,1747,1771,1322,663


In [ ]:
#| export 

def sample_raster_with_polygons(sampling_locations:Path, 
                                input_raster:Path, 
                                target_column:str=None,
                                gpkg_layer:str=None,
                                band_names:list[str]=None,
                                rename_target:str=None,
                                stats:list[str]=['min', 'max', 'mean', 'count'],
                                categorical:bool=False
    ) -> gpd.GeoDataFrame:
    "Extract values from `input_raster` using polygons from `sampling_locations` with `rasterstats.zonal_stats` for all bands"

    if str(sampling_locations).endswith('gpkg') and not gpkg_layer:
        raise Exception(
           '`sampling_locations` is .gpkg but no `gpkg_layer` specified'
        )
    
    gdf = gpd.read_file(sampling_locations, layer=gpkg_layer)
    with rio.open(input_raster) as src:
        if src.gcps[1]: in_crs = src.gcps[1]
        else: in_crs = src.crs
        gdf = gdf.to_crs(in_crs)
        n_bands = src.count
        prof = src.profile
    zstats = []
    for i in range(n_bands):
        zstats.append(zonal_stats(gdf, input_raster, band_num=i+1, categorical=categorical, stats=stats, nodata=-999))

    out_gdf = gdf[[target_column, 'geometry']].copy()
    
    bands = [f'band_{i}' for i in range(n_bands)]

    if band_names:
        assert len(bands) == len(band_names), f'Mismatch provided band names ({len(band_names)}) and number of bands in raster ({len(bands)})'
        bands = band_names

    if target_column in bands:
        if not rename_target:
            raise Exception(
                "One of the band names is the same as target column. Provide rename_target"
            )
        out_gdf.rename(columns={target_column: rename_target}, inplace=True)
    elif rename_target:
        out_gdf.rename(columns={target_column: rename_target}, inplace=True)

    for i, b in enumerate(bands):
        temp = pd.json_normalize(data=zstats[i])
        temp.rename(columns={c: f'{b}_{c}' for c in temp.columns}, inplace=True)
        out_gdf = out_gdf.join(temp)

    return out_gdf

In [ ]:
#| hide

# tests
test_fail(sample_raster_with_polygons, args=('test.gpkg', example_raster, 'id'))
test_fail(sample_raster_with_polygons, args=(example_points, example_raster, 'id', None, ['id']*9))

Example polygons here are previous points buffered by 40 meters.

In [ ]:
out_gdf = sample_raster_with_polygons(example_polys, example_raster, 'id')
out_gdf.iloc[0]

id                                                           10.0
geometry        MULTIPOLYGON (((311800.59915342694 7604880.390...
band_0_min                                                  266.0
band_0_max                                                  415.0
band_0_mean                                            335.708333
band_0_count                                                   48
band_1_min                                                  351.0
band_1_max                                                  696.0
band_1_mean                                              582.1875
band_1_count                                                   48
band_2_min                                                  412.0
band_2_max                                                  699.0
band_2_mean                                            524.520833
band_2_count                                                   48
band_3_min                                                  885.0
band_3_max

As `sample_raster_with_polygons` utilizes `rasterstats.zonal_statistics`, all stats supported by it can be provided with parameter `stats`. More information [here](https://pythonhosted.org/rasterstats/manual.html#zonal-statistics).

In [ ]:
out_gdf = sample_raster_with_polygons(example_polys, example_raster, 'id', stats=['min', 'max', 'sum', 'median', 'range'])
out_gdf.iloc[0]

id                                                            10.0
geometry         MULTIPOLYGON (((311800.59915342694 7604880.390...
band_0_min                                                   266.0
band_0_max                                                   415.0
band_0_sum                                                 16114.0
band_0_median                                                338.0
band_0_range                                                 149.0
band_1_min                                                   351.0
band_1_max                                                   696.0
band_1_sum                                                 27945.0
band_1_median                                                590.0
band_1_range                                                 345.0
band_2_min                                                   412.0
band_2_max                                                   699.0
band_2_sum                                                 251